In [1]:
# Ref
# https://huggingface.co/docs/evaluate/main/en/transformers_integrations
# https://www.kaggle.com/code/fadyelkbeer/mt5-multilingual-xlsum
# https://colab.research.google.com/github/wandb/examples/blob/master/colabs/huggingface/Optimize_Hugging_Face_models_with_Weights_&_Biases.ipynb
# https://docs.wandb.ai/guides/integrations/huggingface

In [2]:
%pip install evaluate rouge_score -q

# Wandb Setup

In [3]:
import wandb
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

# I have saved my API token with "wandb_api" as Label. 
# If you use some other Label make sure to change the same below. 
wandb_api = user_secrets.get_secret("wandb_api") 

wandb.login(key=wandb_api)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Process data function

In [4]:
from transformers import AutoTokenizer
import re
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/mT5_multilingual_XLSum",legacy=False)

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [5]:
def preprocess_function(dataset):
    text_token_len = 512
    sum_token_len = 100
    # Convert the text and summary columns to tensors.
    input_text = dataset["text"]
    target_text = dataset["sum"]

    # this line just follow model card in huggingface
    WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))
    input_text = [WHITESPACE_HANDLER(text) for text in input_text]
    
    # Tokenize the input and target text.
    text_token = tokenizer(input_text, truncation=True, padding=True, max_length=text_token_len)
    sum_token = tokenizer(target_text, truncation=True, padding=True, max_length=sum_token_len)

    # model input
    model_inputs = text_token
    model_inputs["labels"] = sum_token["input_ids"]
    return model_inputs

# Setup evaluation

In [6]:
import nltk
import evaluate
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # decode preds and labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # rougeLSum expects newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return result

# Train model

In [8]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq
from datasets import load_dataset
import numpy as np

In [9]:
model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [10]:
project_name = "text-summarization-model"
use_artifact = "dylanon/text-summarization-model/slices:v2"
train_dataset = "slice_0.parquet"
train_rows = 10000
run_name = "demo"
# with wandb.init(project=project_name) as run:

run = wandb.init(project=project_name, name=run_name, job_type="train")

# load dataset-artifact
artifact = run.use_artifact("dylanon/text-summarization-model/slices:v2", type='dataset')
artifact_dir = artifact.download()
    
# read artifact
dataset = load_dataset('parquet',data_files=(artifact_dir + "/" + train_dataset), split="train[:{}]".format(train_rows))
dataset = dataset.train_test_split(test_size=0.2)
    
# prepare data
tokenized_dataset = dataset.map(preprocess_function, batched=True)
    
# train
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    report_to="wandb",
    auto_find_batch_size = True,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    fp16=True, # this can use with cuda only
    predict_with_generate=True,
)

trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# remove unused variable to free memory
import gc
import torch

del dataset
del tokenized_dataset
gc.collect()
torch.cuda.empty_cache()

# train
trainer.train()

# export model to file
trainer.save_model("./exported_model")


wandb: Currently logged in as: 63050123. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact slices:v2, 6632.04MB. 11 files... 
wandb:   11 of 11 files downloaded.  
Done. 0:0:26.5


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/default-ed68884e7a518a50/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


eval/loss,█▂▁
eval/rouge1,▁▆█
eval/rouge2,▁▆█
eval/rougeL,▁▆█
eval/rougeLsum,▁▇█
eval/runtime,█▂▁
eval/samples_per_second,▁▇█
eval/steps_per_second,▁▇█
train/epoch,▁▁██
train/global_step,▁▃██
train/total_flos,▁


ValueError: too many values to unpack (expected 2)